In [ ]:
# import functions from .py files
import pprint, sqlite3
from SPASE_Scraper_Script import SPASE_Scraper
from PathGrabber import getPaths
from SQLiteFun import (create_tables, add_Metadata, add_Sources)

# list that holds paths returned by PathGrabber
SPASE_paths = []

# print Flag (True prints, False does not)
printFlag = False

# get user input and extract all SPASE records
print("Enter root folder you want to search")
folder = input()
print("You entered " + folder)
SPASE_paths = getPaths(folder, SPASE_paths)
if printFlag:
    print("The number of records is "+ str(len(SPASE_paths)))
    print("The SPASE records found are:")
    print(SPASE_paths)
print("===========================================================================================================")

# list that holds SPASE records already checked
searched = []

# iterate through all SPASE records returned by PathGrabber
for record in SPASE_paths:
    # scrape metadata for each record
    if record not in searched:
        (RID, RIDField, author, authorField, authorRole, pub, pubField, pubDate, pubDateField, datasetName, datasetNameField, 
         desc, descField, PID, PIDField, AccessRights, licenseField, datalinkField) = SPASE_Scraper(record)
        
        # list that holds required fields
        # required = [RID, description, author, authorRole, url]
        
        searched.append(record)
        pubYear = pubDate[0:4]
        author = ", ".join(author)
        authorRole = ", ".join(authorRole)
        if printFlag:
            print("The ResourceID is " + RID + " which was obtained from " + RIDField)
            print("The author(s) are " + author + " who are " + authorRole + " which was obtained from " + authorField)
            print("The publication year is " + pubYear + " which was obtained from " + pubDateField)
            print("The publisher is " + pub + " which was obtained from " + pubField)
            print("The dataset name is " + datasetName + " which was obtained from " + datasetNameField)
            print("The description is " + desc + " which was obtained from " + descField)
            print("The persistent identifier is " + PID + " which was obtained from " + PIDField)
            print("The URLs with their associated product keys obtained from " + datalinkField + """ and their 
                  license(s) obtained from """ + licenseField + " are: ")
            pprint.pprint(AccessRights)

        url = []
        prodKey= []
        license = ""
        desired = False

        # separate license, url, and product keys from AccessRights to store in db
        for k, v in AccessRights.items():
            # check if no URLs returned for any access
            if not v:                
                continue
            else:
                license = k
                # check if any urls are for consideration
                for link in v.keys():
                    if ("nasa.gov" or "virtualsolar.org") in link:
                        desired = True
                for key, val in v.items():
                    # allow desired URLs to be added to database
                    if ("nasa.gov" or "virtualsolar.org") in key:
                        url.append(str(key))
                        if str(val) == "[]":
                            prodKey.append("")
                        else:
                            pKey = str(val).replace("[\'","")
                            pKey = pKey.replace("\']","")
                            prodKey.append(pKey)
                    else:
                        # if no desired URLs present
                        if not desired:
                            url = ["No NASA Links"]
                            prodKey = [""]
                            
                if printFlag:
                    print(k + " was assigned to license")
                    print(str(url) + " was assigned to url")
                    print(str(prodKey) + " was assigned to prodKey")

                    
        # add tables to existing database
        create_tables()
        
        # insert metadata entries into table
        i = 0        
        try:
            with sqlite3.connect('SPASE_Data.db') as conn:
                # add a new SPASE Record
                for urls in url:
                    Metadata = (RID,author,authorRole,pub,pubYear,datasetName,license,url[i],prodKey[i],desc,PID)
                    Record_id = add_Metadata(conn, Metadata)
                    print(f'Created a Metadata entry with the row number {Record_id}')
                    i += 1
                # add a new Source record
                Sources = (RID,authorField,pubField,pubDateField,datasetNameField,licenseField,
                           datalinkField,descField,PIDField)
                Record_id = add_Sources(conn, Sources)
                print(f'Created a Sources entry with row number {Record_id}')

        except sqlite3.Error as e:
            print(e)

        print("===========================================================================================================")
        
    else:
        continue

# call .py file directly from notebook
#%run ./SPASE_test.py {path} in notebook
#import sys
#args = sys.argv
#path = args[1] in source file

# test path : Small = "/home/jovyan/NASA/NumericalData/DE2" or Big = "/home/jovyan/NASA/NumericalData/ACE"
# complex author ex: /home/jovyan/NASA/NumericalData/Cassini/MAG/PT60S.xml 
# OR /home/jovyan/NASA/NumericalData/ACE/Attitude/Definitive/PT1H.xml
# complex URL ex: /home/jovyan/NASA/NumericalData/ACE/CRIS/L2/P1D.xml


In [2]:
from SPASE_Scraper_Script import SPASE_Scraper

SPASE_Scraper.__doc__

'Takes path of a .xml SPASE record file and returns a tuple of values of varying types which hold all \n    desired metadata and the fields they came from. This will collect the desired metadata following the \n    priority rules determined by SPASE record experts. If any desired metadata is not found, the default \n    value assigned is an empty string.\n    \n    :param path: A string of the absolute/relative path of the SPASE record to be scraped.\n    :type path: String\n    :return: A tuple containing the metadata desired and where they were obtained.\n    :rtype: tuple\n    '

In [1]:
import sqlite3
from SQLiteFun import SDAC_records

SDAC_records("1996")

There are 5 records published by SDAC in the year 1996


In [ ]:
from QueryPrinter import Counts, Links
from SQLiteFun import execution, executionALL, add_TestResults, create_tables
import sqlite3

# updates the TestResults column provided for all links provided by a certain test so they have a 1/"True"
def TestUpdate(records, column):
    """Iterates through the parameter records to set the value for each record in the column provided as 1.
    For each record, a SQLite UPDATE statement is made which is then passed to the executionALL function 
    to be executed. The rowNum of the record updated is also collected by calling the execution function.
    
    :param records: A list of the links that pass a specific analysis test.
    :type records: list
    :param column: A string of the TestResults column to be updated.
    :type column: string
    """
    for record in records:
        #print(record + " is the current record")
        Stmt = f""" UPDATE TestResults
                            SET '{column}' = 1
                            WHERE SPASE_id = '{record}' """
        Record_id = execution(f""" SELECT rowNum FROM TestResults WHERE SPASE_id = '{record}';""")
        executionALL(Stmt)
        #print(f"Updated a TestResults entry with the row number {Record_id}")
        #print("===========================================================================")

# print counts of SPASE records that answer analysis questions
#Obj = Counts()
#Obj.allRecords()
#Obj.SDAC_Records()
#Obj.SPDF_Records()

# print SPASE_id's of records that answer analysis questions
testObj = Links()
(authorRecords, pubRecords, pubYrRecords, datasetNameRecords, licenseRecords, urlRecords, NASAurlRecords, 
 PIDRecords, descriptionRecords, citationRecords, complianceRecords) = testObj.allRecords()
#testObj.SDAC_Records()
#testObj.SPDF_Records()

# create the table with 0 as default for all, passing all records to the first insert call
"""
try:
    with sqlite3.connect('SPASE_Data.db') as conn:
        # add a new SPASE Record
        for record in records:
            Test = (record,0,"","",0,0,0,0,0,0,0,0,0,0,0,"")
            Record_id = add_TestResults(conn, Test)
            print(f'Created a TestResults entry with the row number {Record_id}')

except sqlite3.Error as e:
            print(e)"""

# iterate thru lists one by one and update column for each record if in the list (if record in author, has_author = 1)
# UPDATE stmt for each test
TestUpdate(authorRecords, "has_author")    
TestUpdate(pubRecords, "has_pub")
TestUpdate(pubYrRecords, "has_pubYr")
TestUpdate(datasetNameRecords, "has_datasetName")
TestUpdate(licenseRecords, "has_license")
TestUpdate(urlRecords, "has_url")
TestUpdate(NASAurlRecords, "has_NASAurl")
TestUpdate(PIDRecords, "has_PID")
TestUpdate(descriptionRecords, "has_desc")
TestUpdate(citationRecords, "has_citation")
TestUpdate(complianceRecords, "has_compliance")


# TO DO: need to finish coding for Records table